# OpenAI Tool Call

For more information: https://platform.openai.com/docs/guides/function-calling


In [1]:
%pip install pydantic openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def get_weather(location: str) -> str:
    """
    Generate a weather message for a given location.

    Args:
        location (str): The location for which to generate the weather report.

    Returns:
        str: A weather message for the specified location.
    """
    return f"The weather in {location} is sunny with a chance of clouds."

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Generate a weather message for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location for which to generate the weather report.",
                    },
                },
                "required": ["location"],
            },
        },
    },
]

In [5]:
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """
You report the weather of a given location using emojis. 

Use the following format for your answers:
1. Tandil: ☀️ - 🌡
2. London: ️🌧️ - ❄️
"""

messages = [
  {"role": "system", "content": SYSTEM_PROMPT},
  {"role": "user", "content": "What's the weather like in Paris today?"}
]

completion = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    tools=tools
)

In [6]:
print(completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_XlSuBy3G02kt3hgM0PyZWRv8', function=Function(arguments='{"location":"Paris"}', name='get_weather'), type='function')]))]


In [7]:
import json

# Referencias a las tools que queremos exponer al LLM
tools_map = {
    "get_weather": get_weather
}

# Retornos de las tools
tools_returns = []

# Abstracción para llamar a una function dentro de tools_map
def call_function(name, args):
    func = tools_map.get(name)
    if not func:
        raise ValueError(f"Function '{name}' not found.")
    return func(**args)

# Iteramos sobre la lista de tools a llamar
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    result = call_function(name, args)
    tools_returns.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(result)
    })

In [8]:
messages

[{'role': 'system',
  'content': '\nYou report the weather of a given location using emojis. \n\nUse the following format for your answers:\n1. Tandil: ☀️ - 🌡\n2. London: ️🌧️ - ❄️\n'},
 {'role': 'user', 'content': "What's the weather like in Paris today?"}]

In [9]:
# Add the assistant's message with tool calls to the conversation
assistant_message = {
    "role": "assistant",
    "content": completion.choices[0].message.content,
    "tool_calls": completion.choices[0].message.tool_calls
}

# Construct the full conversation: original messages + assistant message + tool responses
msgs = messages + [assistant_message] + tools_returns

# Send back to OpenAI to get the final response
completion = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=msgs,
    tools=tools
)

In [10]:
print(json.dumps(msgs, indent=2, default=str))

[
  {
    "role": "system",
    "content": "\nYou report the weather of a given location using emojis. \n\nUse the following format for your answers:\n1. Tandil: \u2600\ufe0f - \ud83c\udf21\n2. London: \ufe0f\ud83c\udf27\ufe0f - \u2744\ufe0f\n"
  },
  {
    "role": "user",
    "content": "What's the weather like in Paris today?"
  },
  {
    "role": "assistant",
    "content": null,
    "tool_calls": [
      "ChatCompletionMessageFunctionToolCall(id='call_XlSuBy3G02kt3hgM0PyZWRv8', function=Function(arguments='{\"location\":\"Paris\"}', name='get_weather'), type='function')"
    ]
  },
  {
    "role": "tool",
    "tool_call_id": "call_XlSuBy3G02kt3hgM0PyZWRv8",
    "content": "The weather in Paris is sunny with a chance of clouds."
  }
]


In [11]:
print(json.dumps(completion.choices, indent=2, default=str))

[
  "Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Paris: \u2600\ufe0f - \ud83c\udf25\ufe0f', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))"
]


In [14]:
completion.choices[0].message.content

'Paris: ☀️ - 🌥️'